# 02. Реализация и тестирование классификаторов

## Цель
Реализовать и протестировать все три варианта классификаторов рыночных зон:
1. Market Zone Analyzer (MZA) - четырехуровневая система
2. trend_classifier_iziceros - сегментация временных рядов  
3. trading_classifier.pine - полосы тренда

## Задачи
1. Реализовать все три классификатора
2. Протестировать на данных BTC
3. Сравнить результаты
4. Выбрать лучший для дальнейшей работы


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
import os

# Добавляем путь к нашим модулям
sys.path.append('../classifiers')

# Прямой импорт классов
from base_classifier import BaseMarketZoneClassifier
from mza_classifier import MZAClassifier

print("Библиотеки загружены успешно!")
print(f"Текущая директория: {os.getcwd()}")
print(f"Доступные модули: {[m for m in sys.modules.keys() if 'classifier' in m]}")


ModuleNotFoundError: No module named 'classifiers'

In [ ]:
# Загружаем подготовленные данные
print("=== ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ ===")

# Загружаем данные из папки indicators/data_frames
timeframes = ['15m', '30m', '1h', '4h', '1d']
data = {}

for tf in timeframes:
    try:
        file_path = f'../../indicators/data_frames/df_btc_{tf}.csv'
        df = pd.read_csv(file_path)
        df['timestamps'] = pd.to_datetime(df['timestamps'])
        df.set_index('timestamps', inplace=True)
        data[tf] = df
        print(f"✅ {tf}: {len(df)} записей")
    except Exception as e:
        print(f"❌ Ошибка загрузки {tf}: {e}")

print(f"\nЗагружено {len(data)} таймфреймов")
print(f"Доступные таймфреймы: {list(data.keys())}")


In [ ]:
# Тестируем Market Zone Analyzer (MZA) на данных 1h
print("=== ТЕСТИРОВАНИЕ MZA КЛАССИФИКАТОРА ===")

# Выбираем данные 1h для тестирования
test_data = data['1h'].copy()
print(f"Тестовые данные: {len(test_data)} записей")
print(f"Период: {test_data.index[0]} - {test_data.index[-1]}")

# Создаем MZA классификатор
mza = MZAClassifier()

# Обучаем на первых 70% данных
train_size = int(len(test_data) * 0.7)
train_data = test_data.iloc[:train_size]
test_data_sample = test_data.iloc[train_size:]

print(f"Обучающая выборка: {len(train_data)} записей")
print(f"Тестовая выборка: {len(test_data_sample)} записей")

# Обучаем классификатор
mza.fit(train_data)
print("✅ MZA классификатор обучен")


In [ ]:
# Получаем предсказания MZA
print("=== ПРЕДСКАЗАНИЯ MZA ===")

# Предсказываем на тестовых данных
predictions = mza.predict(test_data_sample)
print(f"Получено {len(predictions)} предсказаний")

# Статистика предсказаний
unique, counts = np.unique(predictions, return_counts=True)
print("\n📊 Статистика предсказаний:")
for regime, count in zip(unique, counts):
    regime_name = {-1: 'Медвежий', 0: 'Боковой', 1: 'Бычий'}.get(regime, f'Неизвестный_{regime}')
    percentage = count / len(predictions) * 100
    print(f"  {regime_name}: {count} ({percentage:.1f}%)")

# Получаем детальные скоры
detailed_scores = mza.get_detailed_scores(test_data_sample)
print(f"\n📈 Детальные скоры получены: {detailed_scores.shape}")
print("Первые 5 строк детальных скоров:")
print(detailed_scores.head())


In [ ]:
# Визуализация результатов MZA
print("=== ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ MZA ===")

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# График 1: Цена и предсказания
ax1 = axes[0]
ax1.plot(test_data_sample.index, test_data_sample['close'], 'b-', linewidth=1, alpha=0.7, label='Цена BTC')

# Раскрашиваем по предсказаниям
colors = {-1: 'red', 0: 'gray', 1: 'green'}
for regime in [-1, 0, 1]:
    mask = predictions == regime
    if mask.any():
        regime_name = {-1: 'Медвежий', 0: 'Боковой', 1: 'Бычий'}[regime]
        ax1.scatter(test_data_sample.index[mask], test_data_sample['close'][mask], 
                   c=colors[regime], alpha=0.6, s=20, label=f'{regime_name} ({mask.sum()})')

ax1.set_title('MZA: Цена BTC и предсказания рыночных зон')
ax1.set_ylabel('Цена (USD)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# График 2: Детальные скоры
ax2 = axes[1]
ax2.plot(detailed_scores.index, detailed_scores['trend_score'], 'b-', label='Trend Score', alpha=0.7)
ax2.plot(detailed_scores.index, detailed_scores['momentum_score'], 'g-', label='Momentum Score', alpha=0.7)
ax2.plot(detailed_scores.index, detailed_scores['price_action_score'], 'r-', label='Price Action Score', alpha=0.7)
ax2.plot(detailed_scores.index, detailed_scores['overall_score'], 'k-', linewidth=2, label='Overall Score')

ax2.set_title('MZA: Детальные скоры по компонентам')
ax2.set_ylabel('Скор')
ax2.legend()
ax2.grid(True, alpha=0.3)

# График 3: Распределение скоров
ax3 = axes[2]
ax3.hist(detailed_scores['overall_score'], bins=30, alpha=0.7, color='blue', edgecolor='black')
ax3.axvline(2, color='green', linestyle='--', label='Bullish threshold')
ax3.axvline(-2, color='red', linestyle='--', label='Bearish threshold')
ax3.set_title('MZA: Распределение общих скоров')
ax3.set_xlabel('Overall Score')
ax3.set_ylabel('Частота')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Графики созданы")


In [ ]:
# Простой тест MZA классификатора
print("=== ПРОСТОЙ ТЕСТ MZA ===")

# Создаем простые тестовые данные
test_data_simple = pd.DataFrame({
    'open': [100, 101, 102, 103, 104],
    'high': [101, 102, 103, 104, 105],
    'low': [99, 100, 101, 102, 103],
    'close': [101, 102, 103, 104, 105],
    'volume': [1000, 1100, 1200, 1300, 1400]
})

print("Тестовые данные:")
print(test_data_simple)

try:
    # Создаем MZA классификатор
    mza_test = MZAClassifier()
    print(f"✅ MZA классификатор создан: {mza_test}")
    
    # Обучаем на простых данных
    mza_test.fit(test_data_simple)
    print("✅ MZA классификатор обучен")
    
    # Получаем предсказания
    predictions_test = mza_test.predict(test_data_simple)
    print(f"✅ Предсказания получены: {predictions_test}")
    
    print("🎉 MZA классификатор работает!")
    
except Exception as e:
    print(f"❌ Ошибка в MZA: {e}")
    import traceback
    traceback.print_exc()
